# Y2-k10: Path-Control — Long Damage Exposure (10 epochs at λ=0.20)\n\n**Purpose**: Directly test whether low-λ exposure causes irreversible damage.\n\n- Phase 1: λ=0.60, 3 epochs (form good state)\n- Phase 2: λ=0.20, 10 epochs (damage injection)\n- Phase 3: λ=0.60, 13 epochs (attempted recovery: check at 3ep + extend to 13ep)\n- η = 0.8, r = 5%, SST-2, DistilBERT\n- Seeds: 20\n\n**If Phase 3 fails to recover** → path-dependent trapping confirmed

In [ ]:
!pip install -q transformers datasets accelerate

import os
import json
import time
import copy
import random
import warnings
import logging
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import (
    DistilBertTokenizer, DistilBertForSequenceClassification
)
from datasets import load_dataset

warnings.filterwarnings("ignore")
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu = torch.cuda.get_device_properties(0)
    print(f"GPU: {gpu.name} ({gpu.total_mem/1e9:.1f} GB)" if hasattr(gpu, 'total_mem') else f"GPU: {gpu.name}")


## Config

In [ ]:
CONFIG = {
    "experiment": "exp_Y2k10_path_control_long",
    "experiment_id": "Y2k10",
    "series": "Y",
    "experiment_id_prefix": "Y2k10",
    "dataset": "SST-2",
    "num_classes": 2,
    "model_name": "distilbert-base-uncased",
    "noise_type": "symmetric",
    "noise_rate": 0.8,
    "trusted_ratio": 0.05,
    "phases": [
        {"name": "phase1_good", "lambda": 0.60, "epochs": 3},
        {"name": "phase2_damage", "lambda": 0.20, "epochs": 10},
        {"name": "phase3_recovery", "lambda": 0.60, "epochs": 13},
    ],
    "seeds": list(range(20)),
    "batch_size": 64,
    "learning_rate": 2e-5,
    "weight_decay": 0.01,
    "warmup_ratio": 0.1,
    "max_seq_length": 128,
    "use_bf16": True,
    "max_train_samples": 16000,
    "output_dir": "Y2k10_results",
}

os.makedirs(CONFIG["output_dir"], exist_ok=True)

total_ep = sum(p["epochs"] for p in CONFIG["phases"])
n_runs = len(CONFIG["seeds"])
n_used = min(int(67349 * 0.95), CONFIG["max_train_samples"])
steps_per_epoch = n_used // CONFIG["batch_size"]

print(f"Experiment: {CONFIG['experiment']}")
print(f"Phases:")
for p in CONFIG["phases"]:
    print(f"  {p['name']}: λ={p['lambda']}, {p['epochs']} ep")
print(f"Total epochs per run: {total_ep}")
print(f"Seeds: {n_runs}")
est_per_run = steps_per_epoch * total_ep * 25 / 1000 + total_ep * 5
print(f"Est. per run: ~{est_per_run/60:.1f} min")
print(f"Est. total: ~{n_runs * est_per_run / 3600:.1f} h")


## Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_BASE = "/content/drive/MyDrive/Paper-A_Y_series"
DRIVE_OUTPUT = f"{DRIVE_BASE}/exp_Y2k10_path_control_long"
os.makedirs(DRIVE_OUTPUT, exist_ok=True)
LOCAL_OUTPUT = CONFIG["output_dir"]

def save_to_drive(filename, data):
    for d in [LOCAL_OUTPUT, DRIVE_OUTPUT]:
        with open(os.path.join(d, filename), "w") as f:
            json.dump(data, f, indent=2)

def save_figure_to_drive(fig, filename, dpi=300):
    for d in [LOCAL_OUTPUT, DRIVE_OUTPUT]:
        fig.savefig(os.path.join(d, filename), dpi=dpi, bbox_inches="tight", facecolor="white")

print(f"Local: {LOCAL_OUTPUT}")
print(f"Drive: {DRIVE_OUTPUT}")


## Seed Function

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)


## Load Data

In [ ]:
print("Loading SST-2...")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
raw_dataset = load_dataset("glue", "sst2")

train_texts = list(raw_dataset["train"]["sentence"])
train_labels = list(raw_dataset["train"]["label"])
test_texts = list(raw_dataset["validation"]["sentence"])
test_labels = list(raw_dataset["validation"]["label"])

print(f"Train: {len(train_texts)}, Test: {len(test_texts)}")

print("Tokenizing...")
train_encodings = tokenizer(train_texts, truncation=True, padding="max_length",
                            max_length=CONFIG["max_seq_length"], return_tensors="pt")
test_encodings = tokenizer(test_texts, truncation=True, padding="max_length",
                           max_length=CONFIG["max_seq_length"], return_tensors="pt")
print("Done.")


In [ ]:
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

test_dataset = TextDataset(test_encodings, torch.tensor(test_labels, dtype=torch.long))
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
print(f"Test loader: {len(test_dataset)} samples")


## Noise Preparation

In [ ]:
def prepare_noisy_and_trusted(train_labels, noise_rate, trusted_ratio,
                              num_classes, seed, max_noisy_samples=None):
    rng = np.random.RandomState(seed + 10000)
    n = len(train_labels)
    labels = np.array(train_labels)
    indices = np.arange(n)

    trusted_indices = []
    for c in range(num_classes):
        class_idx = indices[labels == c]
        n_sel = max(1, int(len(class_idx) * trusted_ratio))
        trusted_indices.extend(rng.choice(class_idx, size=n_sel, replace=False))
    trusted_indices = np.array(sorted(trusted_indices))

    noisy_mask = np.ones(n, dtype=bool)
    noisy_mask[trusted_indices] = False
    noisy_indices = indices[noisy_mask]

    noisy_labels = labels.copy()
    n_flip = int(len(noisy_indices) * noise_rate)
    flip_idx = rng.choice(noisy_indices, size=n_flip, replace=False)
    for idx in flip_idx:
        candidates = [c for c in range(num_classes) if c != noisy_labels[idx]]
        noisy_labels[idx] = rng.choice(candidates)

    if max_noisy_samples and len(noisy_indices) > max_noisy_samples:
        noisy_indices = np.sort(rng.choice(noisy_indices, size=max_noisy_samples, replace=False))

    actual = np.mean(noisy_labels[noisy_indices] != labels[noisy_indices])
    print(f"  Trusted: {len(trusted_indices)} | Noisy: {len(noisy_indices)} (η={actual:.3f})")
    return torch.tensor(noisy_labels, dtype=torch.long), trusted_indices, noisy_indices

print("Noise test:")
_, ti, ni = prepare_noisy_and_trusted(train_labels, 0.8, 0.05, 2, 0, 16000)


## Core Functions

In [ ]:
def get_head_param_names(model):
    return [n for n, _ in model.named_parameters() if "classifier" in n or "pre_classifier" in n]

def extract_grads(model):
    return {n: p.grad.clone() for n, p in model.named_parameters() if p.grad is not None}

def normalize_grad_dict(grads):
    flat = torch.cat([g.flatten() for g in grads.values()])
    norm = flat.norm()
    if norm > 0:
        return {n: g / norm for n, g in grads.items()}, norm.item()
    return grads, 0.0

def cosine_sim(grads_a, grads_b, param_names=None):
    keys = [n for n in (param_names or sorted(set(grads_a) & set(grads_b)))
            if n in grads_a and n in grads_b]
    if not keys: return 0.0
    va = torch.cat([grads_a[n].flatten() for n in keys])
    vb = torch.cat([grads_b[n].flatten() for n in keys])
    return F.cosine_similarity(va.unsqueeze(0), vb.unsqueeze(0)).item()

def set_mixed_grad(model, g_struct_n, g_value_n, lam):
    for n, p in model.named_parameters():
        if n in g_struct_n and n in g_value_n:
            p.grad = (1 - lam) * g_struct_n[n] + lam * g_value_n[n]
        elif n in g_struct_n:
            p.grad = (1 - lam) * g_struct_n[n]
        elif n in g_value_n:
            p.grad = lam * g_value_n[n]

@torch.no_grad()
def evaluate(model, test_loader, device):
    model.eval()
    correct = total = 0
    total_loss = 0.0
    n = 0
    for batch in test_loader:
        ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        out = model(input_ids=ids, attention_mask=mask, labels=labels)
        preds = out.logits.argmax(dim=-1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        total_loss += out.loss.item()
        n += 1
    model.train()
    acc = correct / total
    return acc, 1.0 - acc, total_loss / n

print("Core functions defined.")


## Training Function (Multi-Phase)

In [ ]:
def run_path_control(seed, config):
    """
    Execute multi-phase path-control experiment.
    Phase 1: λ_high (good-state formation)
    Phase 2: λ_low (damage injection)
    Phase 3: λ_high (attempted recovery)
    Model and optimizer state carry over between phases.
    """
    phases = config["phases"]
    exp_id = f"{config['experiment_id_prefix']}-{seed:03d}"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    set_seed(seed)
    t_start = time.time()

    # Data preparation
    noisy_labels, trusted_idx, noisy_idx = prepare_noisy_and_trusted(
        train_labels, config["noise_rate"], config["trusted_ratio"],
        config["num_classes"], seed, config.get("max_train_samples")
    )

    noisy_dataset = TextDataset(
        {k: v[noisy_idx] for k, v in train_encodings.items()},
        noisy_labels[noisy_idx]
    )
    trusted_dataset = TextDataset(
        {k: v[trusted_idx] for k, v in train_encodings.items()},
        noisy_labels[trusted_idx]
    )

    struct_loader = DataLoader(noisy_dataset, batch_size=config["batch_size"],
                               shuffle=True, drop_last=True)
    value_loader = DataLoader(trusted_dataset,
                              batch_size=min(config["batch_size"], len(trusted_dataset)),
                              shuffle=True, drop_last=False)

    # Model — fresh init (once)
    set_seed(seed)
    model = DistilBertForSequenceClassification.from_pretrained(
        config["model_name"], num_labels=config["num_classes"]
    ).to(device)
    model.train()
    head_params = get_head_param_names(model)

    # Optimizer — persists across ALL phases
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=config["learning_rate"],
        weight_decay=config["weight_decay"]
    )

    use_amp = config.get("use_bf16", False) and device.type == "cuda"
    amp_dtype = torch.bfloat16 if use_amp else torch.float32

    # Initial eval (before any training)
    acc0, err0, loss0 = evaluate(model, test_loader, device)
    print(f"    [Init] error={err0:.4f}")

    all_epoch_logs = []
    phase_summaries = []
    global_epoch = 0

    for phase_idx, phase in enumerate(phases):
        phase_name = phase["name"]
        lam = phase["lambda"]
        n_epochs = phase["epochs"]
        phase_logs = []

        print(f"    Phase {phase_idx+1} ({phase_name}): λ={lam:.2f}, {n_epochs} epochs")

        for ep in range(n_epochs):
            global_epoch += 1
            value_iter = iter(value_loader)
            epoch_loss_s = 0.0
            epoch_loss_v = 0.0
            epoch_cos_sv = []
            n_batches = 0

            for struct_batch in struct_loader:
                # Struct gradient
                optimizer.zero_grad()
                s_ids = struct_batch["input_ids"].to(device)
                s_mask = struct_batch["attention_mask"].to(device)
                s_labels = struct_batch["labels"].to(device)
                with torch.amp.autocast("cuda", dtype=amp_dtype, enabled=use_amp):
                    out_s = model(input_ids=s_ids, attention_mask=s_mask, labels=s_labels)
                out_s.loss.backward()
                g_struct = extract_grads(model)

                # Value gradient
                optimizer.zero_grad()
                try:
                    vb = next(value_iter)
                except StopIteration:
                    value_iter = iter(value_loader)
                    vb = next(value_iter)

                v_ids = vb["input_ids"].to(device)
                v_mask = vb["attention_mask"].to(device)
                v_labels = vb["labels"].to(device)
                with torch.amp.autocast("cuda", dtype=amp_dtype, enabled=use_amp):
                    out_v = model(input_ids=v_ids, attention_mask=v_mask, labels=v_labels)
                out_v.loss.backward()
                g_value = extract_grads(model)

                # Normalize
                g_struct_n, norm_s = normalize_grad_dict(g_struct)
                g_value_n, norm_v = normalize_grad_dict(g_value)

                c_sv = cosine_sim(g_struct_n, g_value_n, head_params)
                epoch_cos_sv.append(c_sv)

                # Mixed gradient
                optimizer.zero_grad()
                set_mixed_grad(model, g_struct_n, g_value_n, lam)
                optimizer.step()

                epoch_loss_s += out_s.loss.item()
                epoch_loss_v += out_v.loss.item()
                n_batches += 1

            # End of epoch eval
            acc, error, test_loss = evaluate(model, test_loader, device)

            epoch_log = {
                "global_epoch": global_epoch,
                "phase": phase_name,
                "phase_epoch": ep + 1,
                "lambda": lam,
                "test_error": round(error, 6),
                "test_acc": round(acc, 6),
                "test_loss": round(test_loss, 6),
                "train_loss_struct": round(epoch_loss_s / max(n_batches, 1), 6),
                "train_loss_value": round(epoch_loss_v / max(n_batches, 1), 6),
                "mean_cos_sv": round(float(np.mean(epoch_cos_sv)), 6),
                "n_batches": n_batches
            }
            all_epoch_logs.append(epoch_log)
            phase_logs.append(epoch_log)

            print(f"      Epoch {global_epoch} (phase_ep {ep+1}/{n_epochs}): "
                  f"error={error:.4f} cos_sv={epoch_log['mean_cos_sv']:.4f}")

        # Phase summary
        phase_final = phase_logs[-1]["test_error"]
        phase_best = min(l["test_error"] for l in phase_logs)
        phase_summaries.append({
            "phase": phase_name,
            "lambda": lam,
            "epochs": n_epochs,
            "final_error": round(phase_final, 6),
            "best_error": round(phase_best, 6)
        })

    elapsed = time.time() - t_start
    result = {
        "experiment_id": exp_id,
        "experiment": config["experiment"],
        "seed": seed,
        "noise_rate": config["noise_rate"],
        "trusted_ratio": config["trusted_ratio"],
        "phases": [{"name": p["name"], "lambda": p["lambda"], "epochs": p["epochs"]} for p in phases],
        "total_epochs": global_epoch,
        "init_error": round(err0, 6),
        "final_error": round(all_epoch_logs[-1]["test_error"], 6),
        "phase_summaries": phase_summaries,
        "epoch_logs": all_epoch_logs,
        "time_seconds": round(elapsed, 1)
    }
    return result

print("Multi-phase training function defined.")


## Execute

In [ ]:
all_results = []
n_runs = len(CONFIG["seeds"])
t_total = time.time()

print("=" * 70)
print(f"Y2-k10: Path Control (Long Damage) — {n_runs} seeds")
for p in CONFIG["phases"]:
    print(f"  {p['name']}: λ={p['lambda']}, {p['epochs']} ep")
print("=" * 70)

for i, seed in enumerate(CONFIG["seeds"]):
    print(f"\nSeed {seed} ({i+1}/{n_runs})")
    print("-" * 50)
    result = run_path_control(seed, CONFIG)
    all_results.append(result)

    elapsed = time.time() - t_total
    eta = elapsed / (i+1) * (n_runs - i - 1)
    ps = result["phase_summaries"]
    print(f"  Summary: " + " → ".join(f"{p['phase']}={p['final_error']:.4f}" for p in ps))
    print(f"  ({result['time_seconds']:.0f}s) [ETA: {eta/60:.0f}m]")

    save_to_drive(f"Y2k10_seed{seed:02d}.json", result)

total_time = time.time() - t_total
save_to_drive("Y2k10_results.json", all_results)
save_to_drive("Y2k10_config.json", CONFIG)

print(f"\n{'='*70}")
print(f"Y2-k10 COMPLETE: {n_runs} runs in {total_time/60:.1f} min")
for phase_name in ["phase1_good", "phase2_damage", "phase3_recovery"]:
    errs = [p["final_error"] for r in all_results for p in r["phase_summaries"] if p["phase"]==phase_name]
    if errs:
        print(f"  {phase_name}: {np.mean(errs):.4f} ± {np.std(errs):.4f}")
print(f"{'='*70}")


## Analysis

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

path = os.path.join(LOCAL_OUTPUT, "Y2k10_results.json")
if not os.path.exists(path):
    path = os.path.join(DRIVE_OUTPUT, "Y2k10_results.json")
with open(path) as f:
    results = json.load(f)

fig, axes = plt.subplots(1, 3, figsize=(21, 6))

# ---- (A) Individual seed trajectories ----
ax = axes[0]
phase_colors = {"phase1_good": "#2CA02C", "phase2_damage": "#D62728", "phase3_recovery": "#1F77B4"}
phase_boundaries = [3, 13, 26]  # cumulative epoch boundaries

for r in results:
    epochs_all = [l["global_epoch"] for l in r["epoch_logs"]]
    errors_all = [l["test_error"] for l in r["epoch_logs"]]
    ax.plot(epochs_all, errors_all, '-', color="gray", alpha=0.2, linewidth=0.8)

# Mean trajectory
all_trajs = np.array([[l["test_error"] for l in r["epoch_logs"]] for r in results])
means = all_trajs.mean(axis=0)
stds = all_trajs.std(axis=0)
total_ep = all_trajs.shape[1]
epochs_range = np.arange(1, total_ep + 1)

ax.plot(epochs_range, means, 'o-', color="black", linewidth=2.5, markersize=4, label="Mean", zorder=5)
ax.fill_between(epochs_range, means - stds, means + stds, alpha=0.15, color="black")

# Phase boundaries
for b in phase_boundaries[:-1]:
    ax.axvline(b + 0.5, color="orange", ls="--", lw=1.5, alpha=0.7)
ax.axhline(0.5, color="gray", ls=":", alpha=0.4)

ax.text(2, 0.95, "Phase 1\nλ=0.60", ha="center", fontsize=10, color="#2CA02C", fontweight="bold")
ax.text(8, 0.95, "Phase 2\nλ=0.20", ha="center", fontsize=10, color="#D62728", fontweight="bold")
ax.text(20, 0.95, "Phase 3\nλ=0.60", ha="center", fontsize=10, color="#1F77B4", fontweight="bold")

ax.set_xlabel("Global Epoch", fontsize=13)
ax.set_ylabel("Test Error", fontsize=13)
ax.set_title("(A) Y2-k10: Error Trajectory", fontsize=13, fontweight="bold")
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1.0)

# ---- (B) Phase summary (bar chart) ----
ax = axes[1]
phase_names = ["phase1_good", "phase2_damage", "phase3_recovery"]
phase_labels_short = ["Phase 1\n(λ=0.60, 3ep)", "Phase 2\n(λ=0.20, 10ep)", "Phase 3\n(λ=0.60, 13ep)"]
colors_bar = ["#2CA02C", "#D62728", "#1F77B4"]

for pi, (pname, plabel) in enumerate(zip(phase_names, phase_labels_short)):
    errs = [p["final_error"] for r in results for p in r["phase_summaries"] if p["phase"] == pname]
    ax.bar(pi, np.mean(errs), yerr=np.std(errs), color=colors_bar[pi], alpha=0.7,
           edgecolor="black", capsize=8)
    ax.text(pi, np.mean(errs) + np.std(errs) + 0.02, f"{np.mean(errs):.3f}",
            ha="center", fontsize=12, fontweight="bold")

ax.axhline(0.185, color="#FF7F0E", ls="--", lw=2, alpha=0.7, label="X1b (λ=0.60, 3ep) ref")
ax.axhline(0.5, color="gray", ls=":", alpha=0.4)
ax.set_xticks(range(3))
ax.set_xticklabels(phase_labels_short, fontsize=10)
ax.set_ylabel("Final Test Error", fontsize=13)
ax.set_title("(B) Phase-End Errors", fontsize=13, fontweight="bold")
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis="y")
ax.set_ylim(0, 1.0)

# ---- (C) Recovery curve detail (Phase 3 only) ----
ax = axes[2]
p3_start = 3 + 10  # global epoch where phase 3 starts
p3_trajs = all_trajs[:, p3_start:]
p3_means = p3_trajs.mean(axis=0)
p3_stds = p3_trajs.std(axis=0)
p3_epochs = np.arange(1, p3_trajs.shape[1] + 1)

ax.plot(p3_epochs, p3_means, 'o-', color="#1F77B4", linewidth=2.5, markersize=6, label="Phase 3 (λ=0.60)")
ax.fill_between(p3_epochs, p3_means - p3_stds, p3_means + p3_stds, alpha=0.15, color="#1F77B4")

# Phase 1 final as reference
p1_finals = all_trajs[:, 2]  # epoch 3
ax.axhline(np.mean(p1_finals), color="#2CA02C", ls="--", lw=2, label=f"Phase 1 final: {np.mean(p1_finals):.3f}")
ax.axhline(0.185, color="#FF7F0E", ls=":", lw=2, alpha=0.7, label="X1b ref: 0.185")
ax.axhline(0.5, color="gray", ls=":", alpha=0.4)
ax.axvline(3.5, color="red", ls="--", alpha=0.5, label="3 ep checkpoint")

ax.set_xlabel("Phase 3 Epoch", fontsize=13)
ax.set_ylabel("Test Error", fontsize=13)
ax.set_title("(C) Recovery Attempt (Phase 3 Detail)", fontsize=13, fontweight="bold")
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
save_figure_to_drive(fig, "Y2k10_analysis.png")
plt.show()

# Quantitative summary
print("\n" + "=" * 60)
print("Y2-k10 Summary")
print("=" * 60)
for pname in phase_names:
    errs = [p["final_error"] for r in results for p in r["phase_summaries"] if p["phase"] == pname]
    print(f"  {pname}: {np.mean(errs):.4f} ± {np.std(errs):.4f}")

p1_err = np.mean([p["final_error"] for r in results for p in r["phase_summaries"] if p["phase"]=="phase1_good"])
p3_err = np.mean([p["final_error"] for r in results for p in r["phase_summaries"] if p["phase"]=="phase3_recovery"])
recovery_gap = p3_err - p1_err

print(f"\nRecovery gap (Phase3 - Phase1): {recovery_gap:+.4f}")
if recovery_gap > 0.10:
    print("→ CONCLUSION: Low-λ exposure causes LASTING DAMAGE that does not recover")
    print("  This supports PATH-DEPENDENT TRAPPING")
elif recovery_gap < 0.03:
    print("→ CONCLUSION: Full recovery — damage is REVERSIBLE")
    print("  X3's phenomenon may be protocol-specific")
else:
    print("→ CONCLUSION: Partial recovery — some damage persists")


## Save & Download

In [ ]:
print(f"All results saved to: {DRIVE_OUTPUT}")
for f in sorted(os.listdir(DRIVE_OUTPUT)):
    sz = os.path.getsize(os.path.join(DRIVE_OUTPUT, f))
    print(f"  {f} ({sz/1024:.1f} KB)")

import shutil
shutil.make_archive(CONFIG["output_dir"], "zip", LOCAL_OUTPUT)
try:
    from google.colab import files
    files.download(f"{CONFIG['output_dir']}.zip")
except:
    pass
